# AMP Analytics Online

#### Input data path for validation :

In [1]:
val path = "/group/amp_ae_core_content/output/umc_availability/analytics/snapshot/20190208/23-01"

path = /group/amp_ae_core_content/output/umc_availability/analytics/snapshot/20190208/23-01


/group/amp_ae_core_content/output/umc_availability/analytics/snapshot/20190208/23-01

In [2]:
import org.apache.spark.SparkConf
import org.apache.spark.sql.{DataFrame,SparkSession, Column}
import org.apache.spark.sql.functions.{col}

import scala.collection.mutable.ListBuffer

In [3]:
%AddDeps org.vegas-viz vegas_2.11 0.3.9 --transitive
%AddDeps org.vegas-viz vegas-spark_2.11 0.3.9

Marking org.vegas-viz:vegas_2.11:0.3.9 for download
Preparing to fetch from:
-> file:/app/lib/
-> https://artifacts.apple.com/repo1-cache
-> https://artifacts.apple.com/jcenter-cache
-> https://artifacts.apple.com/remote-repos
-> https://artifacts.apple.com/pie-release-local
-> https://artifacts.apple.com/sonatype-oss-releases
-> https://artifacts.apple.com/pie-snapshot-local
-> https://artifacts.apple.com/cie-release-local
-> https://artifacts.apple.com/hubble-release-local
-> https://artifacts.apple.com/iso-release-local
-> https://artifacts.apple.com/iso-snapshot-local
-> https://artifacts.apple.com/libs-release
-> https://artifacts.apple.com/libs-snapshot-local
-> https://artifacts.apple.com/amp-release-local
-> https://artifacts.apple.com/amp-snapshot-local
-> https://artifacts.apple.com/geo-libs-release-local
-> https://artifacts.apple.com/geo-libs-snapshot-local
-> New file at /app/lib/https/artifacts.apple.com/repo1-cache/org/webjars/bower/vega/3.0.0-rc4/vega-3.0.0-rc4.jar
-> N

Waiting for a Spark session to start...

Marking org.vegas-viz:vegas-spark_2.11:0.3.9 for download
Preparing to fetch from:
-> file:/app/lib/
-> https://artifacts.apple.com/repo1-cache
-> https://artifacts.apple.com/jcenter-cache
-> https://artifacts.apple.com/remote-repos
-> https://artifacts.apple.com/pie-release-local
-> https://artifacts.apple.com/sonatype-oss-releases
-> https://artifacts.apple.com/pie-snapshot-local
-> https://artifacts.apple.com/cie-release-local
-> https://artifacts.apple.com/hubble-release-local
-> https://artifacts.apple.com/iso-release-local
-> https://artifacts.apple.com/iso-snapshot-local
-> https://artifacts.apple.com/libs-release
-> https://artifacts.apple.com/libs-snapshot-local
-> https://artifacts.apple.com/amp-release-local
-> https://artifacts.apple.com/amp-snapshot-local
-> https://artifacts.apple.com/geo-libs-release-local
-> https://artifacts.apple.com/geo-libs-snapshot-local
-> New file at /app/lib/https/artifacts.apple.com/repo1-cache/org/vegas-viz/vegas-spark_2.11/0.3.9/vegas-spark_2

In [4]:
import vegas._
import vegas.render.WindowRenderer._
import vegas.data.External._
import vegas.sparkExt._

implicit val render = vegas.render.ShowHTML(kernel.display.content("text/html", _))

render = <function1>


<function1>

In [5]:
val df = spark.read.parquet(path)
df.printSchema()

root
 |-- content_canonical_id: string (nullable = true)
 |-- reference_id: string (nullable = true)
 |-- service_id: string (nullable = true)
 |-- version_id: string (nullable = true)
 |-- offering_type: string (nullable = true)
 |-- offering_type_value: string (nullable = true)
 |-- offering_type_display_value: string (nullable = true)
 |-- start_time: timestamp (nullable = true)
 |-- end_time: timestamp (nullable = true)
 |-- restrictions: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- restrictionType: struct (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |    |    |    |-- displayValue: string (nullable = true)
 |    |    |-- restrictionRelationship: struct (nullable = true)
 |    |    |    |-- type: string (nullable = true)
 |    |    |    |-- value: string (nullable = true)
 |    |    |    |-- displayValue: string (nullable = true)
 |    |    |-- value: string (nulla

df = [content_canonical_id: string, reference_id: string ... 25 more fields]


[content_canonical_id: string, reference_id: string ... 25 more fields]

In [7]:
val totalCount = df.count()
println(s" TOTAL NUMBER OF RECORDS IN DATASET : $totalCount")

 TOTAL NUMBER OF RECORDS IN DATASET : 145994496


totalCount = 145994496


145994496

In [ ]:
val dfColumns = df.columns   
 val lb = ListBuffer[(String, Long, Float)]()
for(colName <- dfColumns){
        val nullCount : Long = df.where(col(colName).isNull).count
        val percentNull = (nullCount.toFloat*100)/totalCount
        val t  = (colName, nullCount, percentNull)
        lb += t
      }

val ll : List[(String, Long, Float)] = lb.toList
val dfCountNull : DataFrame = ll.toDF("colName", "nullCount","percentNull")

dfCountNull.show(30, false)

+-------------------------------------+---------+------------+
|colName                              |nullCount|percentNull |
+-------------------------------------+---------+------------+
|content_canonical_id                 |983911   |0.6739371   |
|reference_id                         |1        |6.8495734E-7|
|service_id                           |1        |6.8495734E-7|
|version_id                           |1        |6.8495734E-7|
|offering_type                        |132289593|90.612724   |
|offering_type_value                  |132289593|90.612724   |
|offering_type_display_value          |132289593|90.612724   |
|start_time                           |132289593|90.612724   |
|end_time                             |132289593|90.612724   |
|restrictions                         |144021404|98.64852    |
|is_live                              |132289593|90.612724   |
|is_broadcast                         |132289593|90.612724   |
|is_premium                           |132289593|90.612

#### Input values for colX, colY (X and Y axis columns for Null %age visualization) 

In [10]:
val colX = "colName"
val colY = "percentNull"

colX = colName
colY = percentNull


percentNull

In [11]:
val usingSparkdf = Vegas("UsingSpark")
  .withDataFrame(dfCountNull)
  .encodeX(colX, Nom, scale=Scale(bandSize = 20.0), hideAxis=false)
  .encodeY(colY, Quant)
.encodeColor(colX, Nom)
.filter("datum.percentNull > 0")
  .mark(Bar)

usingSparkdf.show

<iframe id="frame-vegas-5c80ecda-1e48-4b1e-9abc-1f370565a9cf" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "bar",
 "encoding" : {
 "x" : {
 "axis" : true,
 "scale" : {
 "bandSize" : 20.0
 },
 "field" : "colName",
 "type" : "nominal"
 },
 "y" : {
 "field" : "percentNull",
 "type" : "quantitative"
 },
 "color" : {
 "field" : "colName",
 "type" : "nominal"
 }
 },
 "description" : "UsingSpark",
 "data" : {
 "values" : [
 {
 "colName" : "content_canonical_id",
 "nullCount" : 983911,
 "percentNull" : 0.6739370822906494
 },
 {
 "colName" : "reference_id",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "service_id",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "version_id",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "offering_type",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "offering_type_value",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "offering_type_display_value",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "start_time",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "end_time",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "restrictions",
 "nullCount" : 144021404,
 "percentNull" : 98.64852142333984
 },
 {
 "colName" : "is_live",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "is_broadcast",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "is_premium",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "primary_locale",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "closed_captioning_locales",
 "nullCount" : 141477385,
 "percentNull" : 96.9059829711914
 },
 {
 "colName" : "subtitle_locales",
 "nullCount" : 142684680,
 "percentNull" : 97.73291015625
 },
 {
 "colName" : "dubbed_locales",
 "nullCount" : 145994496,
 "percentNull" : 100.0
 },
 {
 "colName" : "subtitle_deaf_hard_of_hearing_locales",
 "nullCount" : 145425724,
 "percentNull" : 99.61042022705078
 },
 {
 "colName" : "audio_track_locales",
 "nullCount" : 144305714,
 "percentNull" : 98.84325408935547
 },
 {
 "colName" : "audio_track_formats",
 "nullCount" : 138691781,
 "percentNull" : 94.9979476928711
 },
 {
 "colName" : "video_quality",
 "nullCount" : 135014247,
 "percentNull" : 92.47898864746094
 },
 {
 "colName" : "video_color_range",
 "nullCount" : 137186761,
 "percentNull" : 93.96707916259766
 },
 {
 "colName" : "created",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "etl_batch_sk",
 "nullCount" : 0,
 "percentNull" : 0.0
 },
 {
 "colName" : "etl_create_ts",
 "nullCount" : 0,
 "percentNull" : 0.0
 },
 {
 "colName" : "etl_change_ts",
 "nullCount" : 0,
 "percentNull" : 0.0
 },
 {
 "colName" : "etl_source_batch_sk",
 "nullCount" : 0,
 "percentNull" : 0.0
 }
 ]
 },
 "transform" : {
 "filter" : "datum.percentNull > 0"
 }
}
 }
 vg.embed("#vegas-5c80ecda-1e48-4b1e-9abc-1f370565a9cf", embedSpec, function(error, result) {});
 </script>
 <div id='vegas-5c80ecda-1e48-4b1e-9abc-1f370565a9cf'></div>
 </body>
</html>">

usingSparkdf = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(Some(AxisBoolean(true)),Some(Scale(None,None,None,None,Some(BandSizeDouble(20.0)),None,None,None,None,None,None)),None,Some(colName),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(percentNull),Some(Quantitative),None,None,None,None,None)),None,None,Some(ChannelDefWithLegend(None,None,None,Some(colName),Some(Nominal),None,None,None,None,None)),None,None,None,None,None,None,None,None)),None,Some(UsingSpark),Some(Data(None,None,Some(List(Values(Map(colName -> content_canonical_id, nullCount -> 983911, percentNull -> 0.6739371)), Values(Map(colName -> reference_id, nullCount -> 1, percentNull -> 6.84957...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Bar,Some(Encoding(None,None,Some(PositionChannelDef(Some(AxisBoolean(true)),Some(Scale(None,None,None,None,Some(BandSizeDouble(20.0)),None,None,None,None,None,None)),None,Some(colName),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(percentNull),Some(Quantitative),None,None,None,None,None)),None,None,Some(ChannelDefWithLegend(None,None,None,Some(colName),Some(Nominal),None,None,None,None,None)),None,None,None,None,None,None,None,None)),None,Some(UsingSpark),Some(Data(None,None,Some(List(Values(Map(colName -> content_canonical_id, nullCount -> 983911, percentNull -> 0.6739371)), Values(Map(colName -> reference_id, nullCount -> 1, percentNull -> 6.8495734E-7)), Values(Map(colName -> service_id, nullCount -> 1, percentNull -> 6.8495734E-7)), Values(Map(colName -> version_id, nullCount -> 1, percentNull -> 6.8495734E-7)), Values(Map(colName -> offering_type, nullCount -> 132289593, percentNull -> 90.612

In [13]:
val usingSparkdf_circle = Vegas("UsingSpark")
  .withDataFrame(dfCountNull)
  .encodeX(colX, Nom, scale=Scale(bandSize = 20.0), hideAxis=false)
  .encodeY(colY, Quant)
//.encodeColor(colX, Nom)
.filter("datum.percentNull > 0")
  .mark(Circle)

usingSparkdf_circle.show

<iframe id="frame-vegas-7a8da03a-911f-44d2-94b2-fdd13dac1db6" sandbox="allow-scripts allow-same-origin" style="border: none; width: 100%" srcdoc="<html>
 <head>
 <script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/d3/3.5.17/d3.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega/2.6.3/vega.min.js" charset="utf-8"></script>
<script src="https://cdn.jsdelivr.net/webjars/org.webjars.bower/vega-lite/1.2.0/vega-lite.min.js" charset="utf-8"></script>
<script src="https://vega.github.io/vega-editor/vendor/vega-embed.js" charset="utf-8"></script>
 </head>
 <body>
 <script>
 var embedSpec = {
 mode: "vega-lite",
 spec: {
 "mark" : "circle",
 "encoding" : {
 "x" : {
 "axis" : true,
 "scale" : {
 "bandSize" : 20.0
 },
 "field" : "colName",
 "type" : "nominal"
 },
 "y" : {
 "field" : "percentNull",
 "type" : "quantitative"
 }
 },
 "description" : "UsingSpark",
 "data" : {
 "values" : [
 {
 "colName" : "content_canonical_id",
 "nullCount" : 983911,
 "percentNull" : 0.6739370822906494
 },
 {
 "colName" : "reference_id",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "service_id",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "version_id",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "offering_type",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "offering_type_value",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "offering_type_display_value",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "start_time",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "end_time",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "restrictions",
 "nullCount" : 144021404,
 "percentNull" : 98.64852142333984
 },
 {
 "colName" : "is_live",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "is_broadcast",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "is_premium",
 "nullCount" : 132289593,
 "percentNull" : 90.61272430419922
 },
 {
 "colName" : "primary_locale",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "closed_captioning_locales",
 "nullCount" : 141477385,
 "percentNull" : 96.9059829711914
 },
 {
 "colName" : "subtitle_locales",
 "nullCount" : 142684680,
 "percentNull" : 97.73291015625
 },
 {
 "colName" : "dubbed_locales",
 "nullCount" : 145994496,
 "percentNull" : 100.0
 },
 {
 "colName" : "subtitle_deaf_hard_of_hearing_locales",
 "nullCount" : 145425724,
 "percentNull" : 99.61042022705078
 },
 {
 "colName" : "audio_track_locales",
 "nullCount" : 144305714,
 "percentNull" : 98.84325408935547
 },
 {
 "colName" : "audio_track_formats",
 "nullCount" : 138691781,
 "percentNull" : 94.9979476928711
 },
 {
 "colName" : "video_quality",
 "nullCount" : 135014247,
 "percentNull" : 92.47898864746094
 },
 {
 "colName" : "video_color_range",
 "nullCount" : 137186761,
 "percentNull" : 93.96707916259766
 },
 {
 "colName" : "created",
 "nullCount" : 1,
 "percentNull" : 6.849573424005939E-7
 },
 {
 "colName" : "etl_batch_sk",
 "nullCount" : 0,
 "percentNull" : 0.0
 },
 {
 "colName" : "etl_create_ts",
 "nullCount" : 0,
 "percentNull" : 0.0
 },
 {
 "colName" : "etl_change_ts",
 "nullCount" : 0,
 "percentNull" : 0.0
 },
 {
 "colName" : "etl_source_batch_sk",
 "nullCount" : 0,
 "percentNull" : 0.0
 }
 ]
 },
 "transform" : {
 "filter" : "datum.percentNull > 0"
 }
}
 }
 vg.embed("#vegas-7a8da03a-911f-44d2-94b2-fdd13dac1db6", embedSpec, function(error, result) {});
 </script>
 <div id='vegas-7a8da03a-911f-44d2-94b2-fdd13dac1db6'></div>
 </body>
</html>">

usingSparkdf_circle = ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Circle,Some(Encoding(None,None,Some(PositionChannelDef(Some(AxisBoolean(true)),Some(Scale(None,None,None,None,Some(BandSizeDouble(20.0)),None,None,None,None,None,None)),None,Some(colName),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(percentNull),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(UsingSpark),Some(Data(None,None,Some(List(Values(Map(colName -> content_canonical_id, nullCount -> 983911, percentNull -> 0.6739371)), Values(Map(colName -> reference_id, nullCount -> 1, percentNull -> 6.8495734E-7)), Values(Map(colName -> service_id, nullCount -> 1, percentNull -> 6.84957...


ExtendedUnitSpecBuilder(ExtendedUnitSpec(None,None,Circle,Some(Encoding(None,None,Some(PositionChannelDef(Some(AxisBoolean(true)),Some(Scale(None,None,None,None,Some(BandSizeDouble(20.0)),None,None,None,None,None,None)),None,Some(colName),Some(Nominal),None,None,None,None,None)),Some(PositionChannelDef(None,None,None,Some(percentNull),Some(Quantitative),None,None,None,None,None)),None,None,None,None,None,None,None,None,None,None,None)),None,Some(UsingSpark),Some(Data(None,None,Some(List(Values(Map(colName -> content_canonical_id, nullCount -> 983911, percentNull -> 0.6739371)), Values(Map(colName -> reference_id, nullCount -> 1, percentNull -> 6.8495734E-7)), Values(Map(colName -> service_id, nullCount -> 1, percentNull -> 6.8495734E-7)), Values(Map(colName -> version_id, nullCount -> 1, percentNull -> 6.8495734E-7)), Values(Map(colName -> offering_type, nullCount -> 132289593, percentNull -> 90.612724)), Values(Map(colName -> offering_type_value, nullCount -> 132289593, percentNull ->